In [16]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [17]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [18]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([25246858, 1, 53])
Integer dataset size: torch.Size([25246858])


In [19]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size)), :, :]
            targets = intdata[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                
            return inp, target
        except:
            continue
        break

In [20]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        return output, hidden
    

In [21]:
#Define training
def train(epochs):
    #Set start time
    start_time = time.time()
    
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Iterate set of seq_length characters
        for i in range(np.shape(data)[0] - seq_length + 1):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 100 == 0):
                torch.save(model.state_dict(), "charLSTMtrained1024-1.pth")
            
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [22]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 1
seq_length = 75
batch_size = 64

#List of losses
losses = [0]

In [23]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [24]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 21267509


In [25]:
#Train
train(epochs)

Loss: 0.052949 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 15 s
Loss: 0.051680 | ΔLossTotal: -0.0013 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 30 s
Loss: 0.043719 | ΔLossTotal: -0.0092 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 45 s
Loss: 0.045715 | ΔLossTotal: -0.0072 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 01 m 00 s
Loss: 0.038231 | ΔLossTotal: -0.0147 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 01 m 15 s
Loss: 0.038633 | ΔLossTotal: -0.0143 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 01 m 30 s
Loss: 0.038952 | ΔLossTotal: -0.0140 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 01 m 45 s
Loss: 0.038211 | ΔLossTotal: -0.0147 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 02 m 00 s
Loss: 0.037194 | ΔLossTotal: -0.0158 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 02 m 15 s
Loss: 0.036421 | ΔLossTotal: -0.0165 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 02 m 30 s
Loss: 0.036161 | ΔLossTotal: -

Loss: 0.035605 | ΔLossTotal: -0.0173 | Epoch: 01 | Iteration: 0086 | Time elapsed: 00h 21 m 32 s
Loss: 0.035540 | ΔLossTotal: -0.0174 | Epoch: 01 | Iteration: 0087 | Time elapsed: 00h 21 m 47 s
Loss: 0.035532 | ΔLossTotal: -0.0174 | Epoch: 01 | Iteration: 0088 | Time elapsed: 00h 22 m 02 s
Loss: 0.035517 | ΔLossTotal: -0.0174 | Epoch: 01 | Iteration: 0089 | Time elapsed: 00h 22 m 17 s
Loss: 0.035437 | ΔLossTotal: -0.0175 | Epoch: 01 | Iteration: 0090 | Time elapsed: 00h 22 m 32 s
Loss: 0.035445 | ΔLossTotal: -0.0175 | Epoch: 01 | Iteration: 0091 | Time elapsed: 00h 22 m 47 s
Loss: 0.035397 | ΔLossTotal: -0.0176 | Epoch: 01 | Iteration: 0092 | Time elapsed: 00h 23 m 02 s
Loss: 0.035370 | ΔLossTotal: -0.0176 | Epoch: 01 | Iteration: 0093 | Time elapsed: 00h 23 m 17 s
Loss: 0.035346 | ΔLossTotal: -0.0176 | Epoch: 01 | Iteration: 0094 | Time elapsed: 00h 23 m 32 s
Loss: 0.035365 | ΔLossTotal: -0.0176 | Epoch: 01 | Iteration: 0095 | Time elapsed: 00h 23 m 47 s
Loss: 0.035273 | ΔLossTotal: -

Loss: 0.035831 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0171 | Time elapsed: 00h 42 m 50 s
Loss: 0.035910 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0172 | Time elapsed: 00h 43 m 05 s
Loss: 0.035972 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0173 | Time elapsed: 00h 43 m 20 s
Loss: 0.035912 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0174 | Time elapsed: 00h 43 m 35 s
Loss: 0.035892 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0175 | Time elapsed: 00h 43 m 50 s
Loss: 0.035872 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0176 | Time elapsed: 00h 44 m 05 s
Loss: 0.035822 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0177 | Time elapsed: 00h 44 m 20 s
Loss: 0.035872 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0178 | Time elapsed: 00h 44 m 35 s
Loss: 0.035860 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0179 | Time elapsed: 00h 44 m 50 s
Loss: 0.035849 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0180 | Time elapsed: 00h 45 m 05 s
Loss: 0.035778 | ΔLossTotal: -

Loss: 0.036011 | ΔLossTotal: -0.0169 | Epoch: 01 | Iteration: 0256 | Time elapsed: 01h 04 m 08 s
Loss: 0.035957 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0257 | Time elapsed: 01h 04 m 23 s
Loss: 0.035956 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0258 | Time elapsed: 01h 04 m 38 s
Loss: 0.035922 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0259 | Time elapsed: 01h 04 m 53 s
Loss: 0.035845 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0260 | Time elapsed: 01h 05 m 08 s
Loss: 0.035860 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0261 | Time elapsed: 01h 05 m 23 s
Loss: 0.035916 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0262 | Time elapsed: 01h 05 m 38 s
Loss: 0.035943 | ΔLossTotal: -0.0170 | Epoch: 01 | Iteration: 0263 | Time elapsed: 01h 05 m 53 s
Loss: 0.035871 | ΔLossTotal: -0.0171 | Epoch: 01 | Iteration: 0264 | Time elapsed: 01h 06 m 08 s
Loss: 0.035768 | ΔLossTotal: -0.0172 | Epoch: 01 | Iteration: 0265 | Time elapsed: 01h 06 m 23 s
Loss: 0.035773 | ΔLossTotal: -

Loss: 0.034663 | ΔLossTotal: -0.0183 | Epoch: 01 | Iteration: 0341 | Time elapsed: 01h 25 m 26 s
Loss: 0.034654 | ΔLossTotal: -0.0183 | Epoch: 01 | Iteration: 0342 | Time elapsed: 01h 25 m 41 s
Loss: 0.034650 | ΔLossTotal: -0.0183 | Epoch: 01 | Iteration: 0343 | Time elapsed: 01h 25 m 56 s
Loss: 0.034505 | ΔLossTotal: -0.0184 | Epoch: 01 | Iteration: 0344 | Time elapsed: 01h 26 m 11 s
Loss: 0.034411 | ΔLossTotal: -0.0185 | Epoch: 01 | Iteration: 0345 | Time elapsed: 01h 26 m 26 s
Loss: 0.034259 | ΔLossTotal: -0.0187 | Epoch: 01 | Iteration: 0346 | Time elapsed: 01h 26 m 41 s
Loss: 0.034170 | ΔLossTotal: -0.0188 | Epoch: 01 | Iteration: 0347 | Time elapsed: 01h 26 m 56 s
Loss: 0.034186 | ΔLossTotal: -0.0188 | Epoch: 01 | Iteration: 0348 | Time elapsed: 01h 27 m 11 s
Loss: 0.033983 | ΔLossTotal: -0.0190 | Epoch: 01 | Iteration: 0349 | Time elapsed: 01h 27 m 26 s
Loss: 0.033755 | ΔLossTotal: -0.0192 | Epoch: 01 | Iteration: 0350 | Time elapsed: 01h 27 m 41 s
Loss: 0.033812 | ΔLossTotal: -

...

Loss: 0.001423 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4846 | Time elapsed: 20h 52 m 45 s
Loss: 0.001460 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4847 | Time elapsed: 20h 53 m 00 s
Loss: 0.001450 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4848 | Time elapsed: 20h 53 m 15 s
Loss: 0.001435 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4849 | Time elapsed: 20h 53 m 31 s
Loss: 0.001571 | ΔLossTotal: -0.0514 | Epoch: 01 | Iteration: 4850 | Time elapsed: 20h 53 m 46 s
Loss: 0.001472 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4851 | Time elapsed: 20h 54 m 01 s
Loss: 0.001380 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4852 | Time elapsed: 20h 54 m 16 s
Loss: 0.001445 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4853 | Time elapsed: 20h 54 m 31 s
Loss: 0.001301 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4854 | Time elapsed: 20h 54 m 46 s
Loss: 0.001439 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4855 | Time elapsed: 20h 55 m 01 s
Loss: 0.001415 | ΔLossTotal: -

Loss: 0.001266 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 4931 | Time elapsed: 21h 14 m 13 s
Loss: 0.001392 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4932 | Time elapsed: 21h 14 m 28 s
Loss: 0.001381 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4933 | Time elapsed: 21h 14 m 43 s
Loss: 0.001371 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4934 | Time elapsed: 21h 14 m 59 s
Loss: 0.001430 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4935 | Time elapsed: 21h 15 m 14 s
Loss: 0.001289 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 4936 | Time elapsed: 21h 15 m 29 s
Loss: 0.001176 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 4937 | Time elapsed: 21h 15 m 44 s
Loss: 0.001272 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 4938 | Time elapsed: 21h 15 m 59 s
Loss: 0.001361 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 4939 | Time elapsed: 21h 16 m 14 s
Loss: 0.001470 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 4940 | Time elapsed: 21h 16 m 29 s
Loss: 0.001405 | ΔLossTotal: -

Loss: 0.001407 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 5016 | Time elapsed: 21h 35 m 41 s
Loss: 0.001393 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5017 | Time elapsed: 21h 35 m 56 s
Loss: 0.001407 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 5018 | Time elapsed: 21h 36 m 11 s
Loss: 0.001170 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 5019 | Time elapsed: 21h 36 m 27 s
Loss: 0.001283 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5020 | Time elapsed: 21h 36 m 42 s
Loss: 0.001341 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5021 | Time elapsed: 21h 36 m 57 s
Loss: 0.001351 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5022 | Time elapsed: 21h 37 m 12 s
Loss: 0.001319 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5023 | Time elapsed: 21h 37 m 27 s
Loss: 0.001282 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5024 | Time elapsed: 21h 37 m 42 s
Loss: 0.001306 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5025 | Time elapsed: 21h 37 m 58 s
Loss: 0.001318 | ΔLossTotal: -

Loss: 0.001249 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5101 | Time elapsed: 21h 57 m 10 s
Loss: 0.001157 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 5102 | Time elapsed: 21h 57 m 25 s
Loss: 0.001083 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5103 | Time elapsed: 21h 57 m 40 s
Loss: 0.001025 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5104 | Time elapsed: 21h 57 m 55 s
Loss: 0.001009 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5105 | Time elapsed: 21h 58 m 10 s
Loss: 0.000969 | ΔLossTotal: -0.0520 | Epoch: 01 | Iteration: 5106 | Time elapsed: 21h 58 m 25 s
Loss: 0.000996 | ΔLossTotal: -0.0520 | Epoch: 01 | Iteration: 5107 | Time elapsed: 21h 58 m 40 s
Loss: 0.001027 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5108 | Time elapsed: 21h 58 m 55 s
Loss: 0.001042 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5109 | Time elapsed: 21h 59 m 10 s
Loss: 0.001072 | ΔLossTotal: -0.0519 | Epoch: 01 | Iteration: 5110 | Time elapsed: 21h 59 m 25 s
Loss: 0.001206 | ΔLossTotal: -

Loss: 0.001324 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5186 | Time elapsed: 22h 18 m 38 s
Loss: 0.001268 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5187 | Time elapsed: 22h 18 m 53 s
Loss: 0.001153 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 5188 | Time elapsed: 22h 19 m 08 s
Loss: 0.001133 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 5189 | Time elapsed: 22h 19 m 23 s
Loss: 0.001299 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5190 | Time elapsed: 22h 19 m 38 s
Loss: 0.001325 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5191 | Time elapsed: 22h 19 m 53 s
Loss: 0.001349 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5192 | Time elapsed: 22h 20 m 08 s
Loss: 0.001362 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5193 | Time elapsed: 22h 20 m 23 s
Loss: 0.001302 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 5194 | Time elapsed: 22h 20 m 38 s
Loss: 0.001213 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 5195 | Time elapsed: 22h 20 m 54 s
Loss: 0.001107 | ΔLossTotal: -

In [26]:
torch.save(model.state_dict(), "charLSTMtrained1024-1.pth")